In [43]:
import pandas as pd
import numpy as np

import mysql.connector
from sqlalchemy import create_engine
import pandas.io.sql as psql
import sqlalchemy

from dateutil import parser
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta


In [44]:
database_username = 'root'
database_password = 'yash'
database_ip       = '127.0.0.1'
database_name     = 'mainschema'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [45]:
df = pd.read_sql('SELECT * FROM marketsmojorecos', con=database_connection)

In [46]:
start_date = '2016-01-01'
end_date = '2016-12-31'
date_object1 = datetime.strptime(start_date, '%Y-%m-%d').date()
date_object2 = datetime.strptime(end_date, '%Y-%m-%d').date()

In [47]:
mask = (df['predict_date'] > date_object1) & (df['predict_date'] <= date_object2)

# 2016 data

In [48]:
df1 = df.loc[mask]
df1 = df1.reset_index()
del df1['level_0']
del df1['srno']
del df1['index']

In [49]:
print("No of Brokers in 2016 :",df1['broker'].nunique()) 
s1=len(df1)

No of Brokers in 2016 : 61


In [50]:
df3=pd.DataFrame()
for i in range(0,5198): 
    sym=df1.at[i, 'company_name']
    sym = sym.lower()
    sym = sym.replace(" ","")
    sym = sym.replace(".","")
    sym = sym.replace("(i)","")
    sym = sym.replace("(","")
    sym = sym.replace(")","")
    sym = sym.replace("-","")
    sqlrun = "SELECT SYMBOL FROM symbolsmapped WHERE recoName = '"+sym+"'"
    df3 = df3.append(pd.read_sql(sqlrun, con=database_connection),ignore_index=True)

In [51]:
df4 =pd.concat([df1,df3],axis=1)
df4

,broker,company_name,current_price,recomended_buying,predict_date,target_price,SYMBOL
0,KRChoksey,Force Motors,1199.90,3497.00,2016-10-03,4820.00,FORCEMOT
1,AUM Capital,Narayana Hrudayalay,239.00,325.55,2016-09-27,400.00,NH
2,Religare,ICICI Bank,390.15,217.73,2016-08-04,181.82,ICICIBANK
3,Enochventures.com,Parag Milk Foods,137.65,329.6,2016-07-12,425.00,PARAGMILK
4,Edelweiss,Kewal Kiran Clothing,1000.00,1796.00,2016-05-25,2039.00,KKCL
...,...,...,...,...,...,...,...
5193,Axis Direct,Cox & Kings India,3.61,179.00,2016-04-06,338.00,CKFSL
5194,Dolat Capital,Mphasis,966.20,454.00,2016-04-05,620.00,MPHASIS
5195,Angel Broking,Dewan Hsg Fin. Corp.,47.15,189.00,2016-04-05,270.00,None
5196,Angel Broking,MBL Infrastructures,5.57,154.00,2016-04-04,285.00,None


In [52]:
sym=df4.at[1, 'SYMBOL']
sym

'NH'

In [53]:
mydate1=df4.at[1, 'predict_date']#prediction date
print(mydate1)

mydate2 = mydate1 + relativedelta(years=1)#date 1 year after prediction
print(mydate2)

dbyear1 = "nse"+str(mydate1.year)
print(dbyear1)

dbyear2 = "nse"+str(mydate2.year)
print(dbyear2)

2016-09-27
2017-09-27
nse2016
nse2017


In [54]:
sqlrun1 = "SELECT * FROM "+dbyear1+" WHERE Date >= '"+str(mydate1)+"' AND Symbol = '"+sym+"'"
dfprices1 = pd.read_sql(sqlrun1, con=database_connection)
del dfprices1['index']
dfprices1

,Symbol,Date,Closeval
0,NH,2016-09-27,333.45
1,NH,2016-09-28,336.05
2,NH,2016-09-29,323.50
3,NH,2016-09-30,333.55
4,NH,2016-10-03,343.60
...,...,...,...
60,NH,2016-12-26,344.20
61,NH,2016-12-27,338.85
62,NH,2016-12-28,333.80
63,NH,2016-12-29,335.05


In [55]:
sqlrun2 = "SELECT * FROM "+dbyear2+" WHERE Date <= '"+str(mydate2)+"' AND Symbol = '"+sym+"'"
dfprices2 = pd.read_sql(sqlrun2, con=database_connection)
del dfprices2['index']
dfprices2

,Symbol,Date,Closeval
0,NH,2017-01-02,344.35
1,NH,2017-01-03,339.45
2,NH,2017-01-04,334.90
3,NH,2017-01-05,339.65
4,NH,2017-01-06,339.65
...,...,...,...
178,NH,2017-09-21,300.80
179,NH,2017-09-22,299.15
180,NH,2017-09-25,300.35
181,NH,2017-09-26,300.20


In [56]:
dfprices = pd.concat([dfprices1,dfprices2],ignore_index=True)
dfprices

,Symbol,Date,Closeval
0,NH,2016-09-27,333.45
1,NH,2016-09-28,336.05
2,NH,2016-09-29,323.50
3,NH,2016-09-30,333.55
4,NH,2016-10-03,343.60
...,...,...,...
243,NH,2017-09-21,300.80
244,NH,2017-09-22,299.15
245,NH,2017-09-25,300.35
246,NH,2017-09-26,300.20
